# 🪤 Add-ons

## BADA 4 

The OpenAP library can also be used to interact with BADA performance models. 

:::{.callout-important}
In order to user this, you must first obtain a lincense and BADA4 data from EUROCONTROL.
:::

The code for calculating drag, thrust, and fuel models from BADA4 are implemented in OpenAP. The interface is similar to the one used for the open-source OpenAP models. 

To use the BADA4 models, you first need to import the module from addons, and then specify the path to the BADA4 tables on your local system.

```python
from openap.addon import bada4

bada_path = "path/to/bada_4.x/tables"
```

### Drag
In the following example, we show how to use the BADA4 models to calculate the drag of an aircraft at a specific flight condition with clean configuration.

```python
drag = bada4.Drag("A320", bada_path)

drag.clean(mass=60000, tas=300, alt=12_000)  # kg, kt, ft -> N
```

Note that the aircraft type can be ICAO typecode like `A320` or with subtypes like `A320-231`.

### Thrust

To calculate the _maximum_ thrust at climb conditions, you can use the following code:

```python
thrust = bada4.Thrust("A320", bada_path)

thrust.climb(tas=250, alt=10_000)  # kg, kt, ft -> N
```

For max thrust at cruise conditions:

```python
thrust.cruise(tas=350, alt=30_000)
```

For idle thrust:

```python
thrust.idle(tas=250, alt=15_000)
```

### Fuel

The fuel model interface is also similar to OpenAP's own fuel flow model interfaces. An example:

```python
fuel_bada = bada4.FuelFlow("A320", bada_path)

fuel_bada.enroute(mass=60000, tas=350, alt=35_000)  # kg, kt, ft -> kg/s
```

### Vectorized calculations

:::{.callout-tip}
The input parameters can be provided as list or numpy arrays. All the calcualtions are verctorized and hence extremly fast.
:::

In the following example, we show how to calculate the fuel flow from a flight data file obtained from opensky state vectors.


```python
import pandas as pd
import openap
from openap.addon import bada4

typecode = "A319"
fuel_bada = bada4.FuelFlow(typecode, bada_path)

df = pd.read_csv("path/to/your/fightdata.csv")
dt = df.timestamp.diff().bfill().dt.total_seconds()  # time step in seconds

# assume 85% of MTOW as initial mass
mass0 = openap.prop.aircraft(typecode)["mtow"] * 0.85

# first pass to get an initial guess with reference mass
fuel_flow_initial_guess = fuel_bada.enroute(
    mass=mass0,
    tas=df.groundspeed,
    alt=df.altitude,
    vs=df.vertical_rate,
).flatten()

# correct the mass at each time step
mass = mass0 - (fuel_flow_initial_guess * dt).cumsum()

# second pass with corrected mass
fuel_flow = fuel_bada.enroute(
    mass=mass, tas=df.groundspeed, alt=df.altitude, vs=df.vertical_rate
).flatten()

total_fuel = sum(fuel_flow * dt)
```

The cacluation takes approximately `4.6 ms ± 33.7 μs` for a dataframe with ~ 7,000 rows.


In [ ]:
from traffic.core import Traffic, Flight
df = Flight(df).resample("1s").data

dt = df.timestamp.diff().bfill().dt.total_seconds()

In [ ]:
%%timeit
# typecode = "A319"
# assume 85% of MTOW as initial mass
mass0 = openap.prop.aircraft(typecode)["mtow"] * 0.85

# first pass to get an initial guess with reference mass
fuel_flow_initial_guess = fuel_bada.enroute(
    mass=mass0,
    tas=df.groundspeed,
    alt=df.altitude,
    vs=df.vertical_rate,
).flatten()

# correct the mass at each time step
mass = mass0 - (fuel_flow_initial_guess * dt).cumsum()

# second pass with corrected mass
fuel_flow = fuel_bada.enroute(
    mass=mass, tas=df.groundspeed, alt=df.altitude, vs=df.vertical_rate
).flatten()

total_fuel = sum(fuel_flow * dt)